In [1]:
import requests
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [2]:
# Base API URL
BASE_URL = "https://fantasy.premierleague.com/api"

# Helper function to fetch JSON data
def fetch_data(url):
    return requests.get(url).json()

# Fetch data from 2024-2025 season
bootstrap_static_2024 = fetch_data(f"{BASE_URL}/bootstrap-static/")
players_2024 = pd.DataFrame(bootstrap_static_2024['elements'])
teams_2024 = pd.DataFrame(bootstrap_static_2024['teams'])
positions_2024 = pd.DataFrame(bootstrap_static_2024['element_types'])

# Map team and position IDs to readable names
team_map_2024 = teams_2024.set_index('id')['name'].to_dict()
position_map_2024 = positions_2024.set_index('id')['singular_name_short'].to_dict()

# Add readable fields for players
players_2024['team'] = players_2024['team'].map(team_map_2024)
players_2024['position'] = players_2024['element_type'].map(position_map_2024)
players_2024['full_name'] = players_2024['first_name'] + " " + players_2024['second_name']

# Fetch player performance for 2023-2024
gameweek_data_2023 = []
for player_id in players_2024['id']:
    player_data = fetch_data(f"{BASE_URL}/element-summary/{player_id}/")
    history = player_data.get('history_past', [])
    
    # Extract data for 2023-2024 season
    season_2023 = next((season for season in history if season['season_name'] == '2023/24'), None)
    if season_2023:
        gameweek_data_2023.append({
            'id': player_id,
            'full_name': players_2024.loc[players_2024['id'] == player_id, 'full_name'].values[0],
            'team': players_2024.loc[players_2024['id'] == player_id, 'team'].values[0],
            'position': players_2024.loc[players_2024['id'] == player_id, 'position'].values[0],
            'total_points': season_2023['total_points'],
            'matches_played': season_2023['minutes'] // 90,  # Estimate games played
            'value': players_2024.loc[players_2024['id'] == player_id, 'now_cost'].values[0] / 10  # Value in millions
        })

# Convert to DataFrame
df_2023 = pd.DataFrame(gameweek_data_2023)

# Calculate PPM (Points Per Million)
df_2023['ppm'] = df_2023['value'] / df_2023['total_points']
df_2023['points_per_match'] = df_2023['total_points'] / df_2023['matches_played']

df_2023.to_csv('FPL 2023-2024 Player Performance.csv', index=False)

df_2023

,id,full_name,team,position,total_points,matches_played,value,ppm,points_per_match
0,1,Fábio Ferreira Vieira,Arsenal,MID,24,3,5.4,0.225000,8.000000
1,2,Gabriel Fernando de Jesus,Arsenal,FWD,85,16,6.8,0.080000,5.312500
2,3,Gabriel dos Santos Magalhães,Arsenal,DEF,149,33,6.2,0.041611,4.515152
3,4,Kai Havertz,Arsenal,FWD,180,29,7.9,0.043889,6.206897
4,5,Karl Hein,Arsenal,GKP,0,0,4.0,inf,NaN
...,...,...,...,...,...,...,...,...,...
492,559,Nélson Cabral Semedo,Wolves,DEF,63,34,4.5,0.071429,1.852941
493,562,Daniel Castelo Podence,Wolves,MID,0,0,5.5,inf,NaN
494,564,Santiago Bueno,Wolves,DEF,13,9,4.3,0.330769,1.444444
495,565,Pablo Sarabia,Wolves,MID,99,19,5.1,0.051515,5.210526


In [3]:
# Filter the dataframe for players with more than 20 matches
filtered_df = df_2023[df_2023['matches_played'] > 20]

# Extract the top 10 players for each position based on points per match
top_10_per_position = (
    filtered_df.sort_values(by='points_per_match', ascending=False)
    .groupby('position')
    .head(10)
)

# Keep only the relevant columns and order the output
top_10_per_position = top_10_per_position[['full_name', 'position', 'team', 'points_per_match', 'value']]
top_10_per_position = top_10_per_position.sort_values(by=['position', 'points_per_match'], ascending=[True, False])
top_10_per_position

,full_name,position,team,points_per_match,value
19,Benjamin White,DEF,Arsenal,5.515152,6.1
266,Trent Alexander-Arnold,DEF,Liverpool,5.304348,7.0
303,Joško Gvardiol,DEF,Man City,4.920000,6.1
370,Kieran Trippier,DEF,Newcastle,4.625000,5.6
2,Gabriel dos Santos Magalhães,DEF,Arsenal,4.515152,6.2
294,Nathan Aké,DEF,Man City,4.500000,5.3
14,William Saliba,DEF,Arsenal,4.315789,6.2
73,Marcos Senesi,DEF,Bournemouth,4.250000,4.8
293,Manuel Akanji,DEF,Man City,4.148148,5.3
315,Kyle Walker,DEF,Man City,4.100000,5.2


In [4]:
# Create the optimization problem
prob = LpProblem("FPL_Team_Selection", LpMaximize)

# Create a binary variable for each player (1 if selected, 0 otherwise)
player_vars = {i: LpVariable(f"player_{i}", cat='Binary') for i in top_10_per_position.index}

# Objective function: Maximize total points per match (PPM)
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'points_per_match'] for i in top_10_per_position.index)

# Constraints

# Total budget <= 100
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'value'] for i in top_10_per_position.index) <= 100

# Total players = 15
prob += lpSum(player_vars[i] for i in top_10_per_position.index) == 15

# Position constraints
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'GKP') == 2
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'DEF') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'MID') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'FWD') == 3

# Solve the problem
prob.solve()

# Extract the selected players
selected_players = top_10_per_position.loc[[i for i in top_10_per_position.index if player_vars[i].value() == 1]]

# Order by position and points per match
selected_players = selected_players.sort_values(by=['position', 'points_per_match'], ascending=[True, False])

selected_players

,full_name,position,team,points_per_match,value
19,Benjamin White,DEF,Arsenal,5.515152,6.1
303,Joško Gvardiol,DEF,Man City,4.920000,6.1
370,Kieran Trippier,DEF,Newcastle,4.625000,5.6
294,Nathan Aké,DEF,Man City,4.500000,5.3
73,Marcos Senesi,DEF,Bournemouth,4.250000,4.8
354,Alexander Isak,FWD,Newcastle,6.880000,8.6
188,Jean-Philippe Mateta,FWD,Crystal Palace,6.600000,7.2
270,Darwin Núñez Ribeiro,FWD,Liverpool,6.000000,7.1
11,David Raya Martin,GKP,Arsenal,4.218750,5.5
447,Alphonse Areola,GKP,West Ham,3.866667,4.2


In [5]:
selected_players['points_per_match'].sum()

86.03447369973372

In [6]:
selected_players['value'].sum()

99.99999999999999